In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import acquire
import wrangle

In [2]:
df = acquire.get_zillow_data()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    52441 non-null  int64  
 1   parcelid                      52441 non-null  int64  
 2   bedroomcnt                    52441 non-null  float64
 3   bathroomcnt                   52441 non-null  float64
 4   calculatedfinishedsquarefeet  52359 non-null  float64
 5   taxvaluedollarcnt             52440 non-null  float64
 6   yearbuilt                     52325 non-null  float64
 7   taxamount                     52437 non-null  float64
 8   fips                          52441 non-null  float64
 9   assessmentyear                52441 non-null  float64
 10  landtaxvaluedollarcnt         52440 non-null  float64
 11  lotsizesquarefeet             52072 non-null  float64
 12  latitude                      52441 non-null  float64
 13  l

In [4]:
df = wrangle.wrangle_zillow(df)

In [5]:
df = wrangle.wrangle_locs(df)

In [6]:
zips = acquire.get_zillow_zips()

In [7]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

In [8]:
import sklearn.preprocessing

In [9]:
zips = zips.dropna()

In [10]:
zips.zips.value_counts()

91011         2923
92584         1383
92585         1052
90805          954
92555          935
              ... 
93030-6506       3
90815            3
90704            2
93560            2
92560            1
Name: zips, Length: 102, dtype: int64

In [11]:
zips.zips.dtypes

dtype('O')

In [12]:
zips['zips'] = zips['zips'].apply(lambda x: str(x[:5]))

In [13]:
zips['zips'] = zips.zips.astype(int)

In [14]:
zip_counts = zips['zips'].value_counts()

zips = zips[~zips['zips'].isin(zip_counts[zip_counts < 100].index)]

In [15]:
other = acquire.get_zillow_other()

In [16]:
other = other.fillna(0)
other

,Unnamed: 0,parcelid,fireplacecnt,garagecarcnt,poolcnt
0,0,14297519,0.0,2.0,0.0
1,1,17052889,1.0,1.0,0.0
2,2,14186244,0.0,2.0,1.0
3,3,12177905,0.0,0.0,1.0
4,4,12095076,0.0,0.0,1.0
...,...,...,...,...,...
52436,52436,12412492,0.0,0.0,0.0
52437,52437,11000655,0.0,0.0,0.0
52438,52438,17239384,1.0,2.0,0.0
52439,52439,12773139,0.0,0.0,0.0


In [17]:
zips = zips.merge(other, how='inner')

In [18]:
zips

,Unnamed: 0,parcelid,bedrooms,bathrooms,home_size,value,year,tax,fips,land_value,lot_size,county,latitude,longitude,KM_label,zips,fireplacecnt,garagecarcnt,poolcnt
0,50339,12435056,3.0,2.0,1547,91062,1953,1329.02,6037,33520,4982,los_angeles,33.821634,-118.180944,7,90807,0.0,0.0,0.0
1,28375,12114836,5.0,2.0,2238,294404,1910,3702.98,6037,166496,10913,los_angeles,34.160779,-118.181243,2,91105,0.0,0.0,0.0
2,46916,12350784,5.0,5.0,4642,684719,1953,8354.42,6037,173524,7605,los_angeles,33.940562,-118.181111,6,90280,0.0,0.0,0.0
3,20158,14262921,3.0,2.0,1356,180208,1964,2277.76,6059,97817,6002,orange,33.721196,-117.178914,11,92585,0.0,2.0,0.0
4,18052,14185319,3.0,2.0,1619,442099,1968,5074.14,6059,349658,6200,orange,33.868299,-117.178110,14,92551,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25651,27032,14494065,3.0,2.5,2133,810000,1977,8144.50,6059,657679,5782,orange,33.636617,-117.176476,5,92584,0.0,2.0,0.0
25652,42940,10996357,4.0,3.0,2045,519807,1968,6429.02,6037,340413,6009,los_angeles,34.264051,-118.182954,9,91011,0.0,0.0,0.0
25653,46851,11279183,2.0,2.0,1093,163100,1955,2703.78,6037,41100,6983,los_angeles,34.690405,-118.181527,1,93536,0.0,0.0,0.0
25654,44865,14207326,4.0,3.0,2824,620304,1998,7927.78,6059,293459,10402,orange,33.871625,-117.177743,14,92551,0.0,2.0,0.0


In [19]:
from sklearn.impute import SimpleImputer

In [20]:
dummy = pd.get_dummies(zips[['zips', 'fireplacecnt']])

In [21]:
zips = pd.concat([dummy, zips], axis=1)

In [22]:
zips

,zips,fireplacecnt,Unnamed: 0,parcelid,bedrooms,bathrooms,home_size,value,year,tax,...,land_value,lot_size,county,latitude,longitude,KM_label,zips,fireplacecnt,garagecarcnt,poolcnt
0,90807,0.0,50339,12435056,3.0,2.0,1547,91062,1953,1329.02,...,33520,4982,los_angeles,33.821634,-118.180944,7,90807,0.0,0.0,0.0
1,91105,0.0,28375,12114836,5.0,2.0,2238,294404,1910,3702.98,...,166496,10913,los_angeles,34.160779,-118.181243,2,91105,0.0,0.0,0.0
2,90280,0.0,46916,12350784,5.0,5.0,4642,684719,1953,8354.42,...,173524,7605,los_angeles,33.940562,-118.181111,6,90280,0.0,0.0,0.0
3,92585,0.0,20158,14262921,3.0,2.0,1356,180208,1964,2277.76,...,97817,6002,orange,33.721196,-117.178914,11,92585,0.0,2.0,0.0
4,92551,0.0,18052,14185319,3.0,2.0,1619,442099,1968,5074.14,...,349658,6200,orange,33.868299,-117.178110,14,92551,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25651,92584,0.0,27032,14494065,3.0,2.5,2133,810000,1977,8144.50,...,657679,5782,orange,33.636617,-117.176476,5,92584,0.0,2.0,0.0
25652,91011,0.0,42940,10996357,4.0,3.0,2045,519807,1968,6429.02,...,340413,6009,los_angeles,34.264051,-118.182954,9,91011,0.0,0.0,0.0
25653,93536,0.0,46851,11279183,2.0,2.0,1093,163100,1955,2703.78,...,41100,6983,los_angeles,34.690405,-118.181527,1,93536,0.0,0.0,0.0
25654,92551,0.0,44865,14207326,4.0,3.0,2824,620304,1998,7927.78,...,293459,10402,orange,33.871625,-117.177743,14,92551,0.0,2.0,0.0


In [23]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

In [24]:
X_train = pd.DataFrame(zips.drop(columns=['Unnamed: 0', 'parcelid', 'latitude', 'longitude', 'KM_label', 'zips', 'fips','tax', 'value', 'land_value', 'county', 'poolcnt', 'garagecarcnt']))
y_train = pd.DataFrame(zips['value'])

In [25]:
X_train

,fireplacecnt,bedrooms,bathrooms,home_size,year,lot_size,fireplacecnt
0,0.0,3.0,2.0,1547,1953,4982,0.0
1,0.0,5.0,2.0,2238,1910,10913,0.0
2,0.0,5.0,5.0,4642,1953,7605,0.0
3,0.0,3.0,2.0,1356,1964,6002,0.0
4,0.0,3.0,2.0,1619,1968,6200,0.0
...,...,...,...,...,...,...,...
25651,0.0,3.0,2.5,2133,1977,5782,0.0
25652,0.0,4.0,3.0,2045,1968,6009,0.0
25653,0.0,2.0,2.0,1093,1955,6983,0.0
25654,0.0,4.0,3.0,2824,1998,10402,0.0


In [26]:
y_train['pred_mean'] = y_train.value.mean()
#y_validate['pred_mean'] = y_validate.value.mean()

In [27]:
y_train['pred_median'] = y_train.value.median()
#y_validate['pred_median'] = y_train.value.median()

In [28]:
X_train

,fireplacecnt,bedrooms,bathrooms,home_size,year,lot_size,fireplacecnt
0,0.0,3.0,2.0,1547,1953,4982,0.0
1,0.0,5.0,2.0,2238,1910,10913,0.0
2,0.0,5.0,5.0,4642,1953,7605,0.0
3,0.0,3.0,2.0,1356,1964,6002,0.0
4,0.0,3.0,2.0,1619,1968,6200,0.0
...,...,...,...,...,...,...,...
25651,0.0,3.0,2.5,2133,1977,5782,0.0
25652,0.0,4.0,3.0,2045,1968,6009,0.0
25653,0.0,2.0,2.0,1093,1955,6983,0.0
25654,0.0,4.0,3.0,2824,1998,10402,0.0


In [29]:
rmse_train = mean_squared_error(y_train.value, y_train.pred_mean)**(1/2)
#rmse_validate = mean_squared_error(y_validate.value, y_validate.pred_mean)**(1/2)

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2))
      #, 
      #"\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

RMSE using Mean
Train/In-Sample:  579497.98


In [30]:
rmse_train = mean_squared_error(y_train.value, y_train.pred_median)**(1/2)
#rmse_validate = mean_squared_error(y_validate.value, y_validate.pred_median)**(1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train, 2))
      #, 
      #"\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

RMSE using Median
Train/In-Sample:  596391.17


In [31]:
lm = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm.fit(X_train, y_train.value)

# predict train
y_train['value_predict_lm'] = lm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.value, y_train.value_predict_lm)**(1/2)

# predict validate
#y_validate['value_predict_lm'] = lm.predict(X_validate)

# evaluate: rmse
#rmse_validate = mean_squared_error(y_validate.value, y_validate.value_predict_lm)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train)
#, 
     # "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for OLS using LinearRegression
Training/In-Sample:  442666.15559283056


lars = LassoLars(alpha=1.0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lars.fit(X_train, y_train.value)

# predict train
y_train['pred_lars'] = lars.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.value, y_train.pred_lars)**(1/2)

# predict validate
#y_validate['pred_lars'] = lars.predict(X_validate)

# evaluate: rmse
#rmse_validate = mean_squared_error(y_validate.value, y_validate.pred_lars)**(1/2)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train)#, 
      #"\nValidation/Out-of-Sample: ", rmse_validate)

glm = TweedieRegressor(power=1, alpha=6)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
glm.fit(X_train, y_train.value)

# predict train
y_train['pred_value_glm'] = glm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.value, y_train.pred_value_glm)**(1/2)

# predict validate
#y_validate['pred_value_glm'] = glm.predict(X_validate)

# evaluate: rmse
#rmse_validate = mean_squared_error(y_validate.value, y_validate.pred_value_glm)**(1/2)

print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train)#, 
    #  "\nValidation/Out-of-Sample: ", rmse_validate)

In [32]:
from sklearn.linear_model import LinearRegression


# compute predictions and add to original dataframe
zips['yhat'] = lm.predict(X_train).round()

zips

,zips,fireplacecnt,Unnamed: 0,parcelid,bedrooms,bathrooms,home_size,value,year,tax,...,lot_size,county,latitude,longitude,KM_label,zips,fireplacecnt,garagecarcnt,poolcnt,yhat
0,90807,0.0,50339,12435056,3.0,2.0,1547,91062,1953,1329.02,...,4982,los_angeles,33.821634,-118.180944,7,90807,0.0,0.0,0.0,413431.0
1,91105,0.0,28375,12114836,5.0,2.0,2238,294404,1910,3702.98,...,10913,los_angeles,34.160779,-118.181243,2,91105,0.0,0.0,0.0,561391.0
2,90280,0.0,46916,12350784,5.0,5.0,4642,684719,1953,8354.42,...,7605,los_angeles,33.940562,-118.181111,6,90280,0.0,0.0,0.0,1719600.0
3,92585,0.0,20158,14262921,3.0,2.0,1356,180208,1964,2277.76,...,6002,orange,33.721196,-117.178914,11,92585,0.0,2.0,0.0,307859.0
4,92551,0.0,18052,14185319,3.0,2.0,1619,442099,1968,5074.14,...,6200,orange,33.868299,-117.178110,14,92551,0.0,2.0,0.0,398041.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25651,92584,0.0,27032,14494065,3.0,2.5,2133,810000,1977,8144.50,...,5782,orange,33.636617,-117.176476,5,92584,0.0,2.0,0.0,629620.0
25652,91011,0.0,42940,10996357,4.0,3.0,2045,519807,1968,6429.02,...,6009,los_angeles,34.264051,-118.182954,9,91011,0.0,0.0,0.0,558734.0
25653,93536,0.0,46851,11279183,2.0,2.0,1093,163100,1955,2703.78,...,6983,los_angeles,34.690405,-118.181527,1,93536,0.0,0.0,0.0,353542.0
25654,92551,0.0,44865,14207326,4.0,3.0,2824,620304,1998,7927.78,...,10402,orange,33.871625,-117.177743,14,92551,0.0,2.0,0.0,773476.0


In [33]:
zips['yhat_baseline'] = zips.value.mean()

In [34]:
zips['residual'] = zips['yhat'] - zips['value']
zips['residual_baseline'] = zips['yhat_baseline'] - zips['value']

In [35]:
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression 
from math import sqrt

In [36]:
zips['residual^2'] = zips.residual ** 2
zips['residual_baseline^2'] = zips.residual_baseline ** 2

In [37]:
SSE = sum(zips['residual^2'])
SSE_baseline = sum(zips['residual_baseline^2'])

print("SSE = ", SSE)
print("SSE - baseline = ", SSE_baseline)

SSE =  5027378512344288.0
SSE - baseline =  8615744150460866.0


In [38]:
MSE = SSE/len(zips)
MSE_baseline = SSE_baseline/len(zips)

print("MSE = ", MSE)
print("MSE baseline = ", MSE_baseline)

MSE =  195953325239.48737
MSE baseline =  335817904211.914


In [39]:
RMSE = sqrt(MSE)
RMSE_baseline = sqrt(MSE_baseline)

print("RMSE = ", RMSE)
print("RMSE baseline = ", RMSE_baseline)

RMSE =  442666.1555161941
RMSE baseline =  579497.97602055
